<a href="https://colab.research.google.com/github/kerupinen/Compdb_final_repo/blob/main/Final_Project_Rupinen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing and Connecting to Database

In [2]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 15.1 MB/s eta 0:00:00


In [3]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient(host='3.230.203.12',  # host is the hostname for the database
                                port=27018,  # port is the port number that mongo is running on
                                username='compdb_student',  # username for the db
                                password='compdbpass',  # password for the db
                                authSource='admin')  # Since our user only exists for the emse6992 db, we need to specify this

# Setup collections and connections
db = client.twitter
stats_coll = db.twitter_statuses
retweets_coll = db.twitter_retweets
favs_coll = db.twitter_favorites
lists_coll = db.twitter_lists_updated
friends_coll = db.twitter_friends_updated


In [4]:
import psycopg2
import psycopg2.extras
import pandas as pd

# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

#Work

##Tweets per weekday

In [5]:
x = 1
weekdayCountList=[]
total = 0
ototal = 0
#While the weekday is 1-7 (Monday being 1 and Sunday being 7)
while x < 8:
    #Select number of tweets tweeted where weekday = x
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #Add number of tweets for each weekday with the number indicating weekday to list
    weekdayCountList.append([x,res[0]['count']])
    #Update day + 1
    x=x+1


print(weekdayCountList)


[[1, 6618], [2, 6264], [3, 6684], [4, 6647], [5, 6729], [6, 5172], [7, 4717]]


In [6]:
import plotly.express as px
#Plot in a bar chart tweets by weekday (Monday being 1 and Sunday being 7)
fig = px.bar(weekdayCountList, x=0, y=1, title = "Tweets by weekday",labels={'0':'Day of Week','1':'Number of Tweets'}, range_y=(4000,7000))
fig.show()

##Retweets and Likes per weekday

In [7]:
x = 1
weekDayTotals=[]
total = 0
ototal = 0
#While the weekday is 1-7 (Monday being 1 and Sunday being 7)
while x < 8:
    #LIKES:
    #Select favortites_count from each tweet where weekday = x
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #for each favorites count, add to total
    for i in res:
        total = total+i['favorites_count']

    #RETWEETS:
    #Select retweet_count from each tweet where weekday = x
    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #for each retweet count, add to total for that weekday
    for w in res:
        ototal = ototal+w['retweet_count']
    #Add weekday,like total, and retweet total per day to list
    weekDayTotals.append([x,total,ototal])
    #update weekday + 1
    x=x+1
    #reset totals for next day
    ototal=0
    total=0


print(weekDayTotals)

[[1, 18805170, 8095532], [2, 20967480, 8806093], [3, 22649066, 11023208], [4, 21073340, 11227533], [5, 28259773, 11119761], [6, 20573004, 16762835], [7, 22489203, 8907312]]


In [8]:
retweetday = []
likesday = []
x=1
#Seperating list into retweets per day and likes per day
for i in weekDayTotals:
    #Taking retweets
    retweetday.append([x,i[2]])
    #Taking likes
    likesday.append([x,i[1]])

    x=x+1

In [9]:
#Charting retweets by weekday (Monday being 1 and Sunday being 7)
fig = px.bar(retweetday, x=0, y=1, title = "retweets by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,20000000))
fig.show()

In [10]:
#Charting Likes By Weekday (Monday being 1 and Sunday being 7)
fig = px.bar(likesday, x=0, y=1, title = "Likes by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,30000000))
fig.show()

##Retweets per Tweet and Likes per Tweet per weekday

In [11]:
x = 1
avgRetweetpd = []
avgLikespd = []
#for each favorites count, add to total
while x < 8:
    #Take the total RETWEETS of the weekday and / by total count of tweets from that weekday
    average = (weekDayTotals[x-1][2])/(weekdayCountList[x-1][1])
    #Adding to a list of retweets per tweet per day
    avgRetweetpd.append([x,average])
    #Take the total LIKES of the weekday and / by total count of tweets from that weekday
    average = (weekDayTotals[x-1][1])/(weekdayCountList[x-1][1])
    #Adding to a list of likes per tweet per day
    avgLikespd.append([x,average])
    #Update weekday+1
    x=x+1

print(avgRetweetpd)
print(avgLikespd)


[[1, 1223.25959504382], [2, 1405.8258301404853], [3, 1649.1932974266906], [4, 1689.112832856928], [5, 1652.513152028533], [6, 3241.074052590874], [7, 1888.3425906296375]]
[[1, 2841.5185856754306], [2, 3347.2988505747126], [3, 3388.549670855775], [4, 3170.3535429517074], [5, 4199.698766532917], [6, 3977.7656612529004], [7, 4767.691965232139]]


Retweet per tweet

In [12]:
#Chart retweet per tweet by weekday
fig = px.bar(avgRetweetpd, x=0, y=1, title = "retweet per tweet by weekday",labels={'0':'Day of Week','1':'Retweets per Tweet'}, range_y=(0,4000))
fig.show()

Like Per Tweet

In [13]:
#Charting Likes Per Tweet By Weekday
fig = px.bar(avgLikespd, x=0, y=1, title = "like per tweet by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,5000))
fig.show()

##Tweets per Week of Month

In [14]:
tweetsPerWeek = []
x=0
y=7
i=1
#While the latest day is less than 36
while y<36:
    #Select distinct number of tweets between (startdate) and (end date) of the week of the month
    query = "SELECT count(distinct(status_id)) FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #Add number of tweets to list of tweets per week
    tweetsPerWeek.append([i,res[0]['count']])
    #Update x to the next week
    x=y+1
    #If in the third week, add 10 to catch all values possible in the last week
    if(y==21):
        y=y+10
    else:
    #Otherwise add 7 to get to the next week
        y=y+7
    #Update i to show next week
    i=i+1

print(tweetsPerWeek)



[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [15]:
#Bar chart of tweets organized by week of the month (1-4)
fig = px.bar(tweetsPerWeek, x=0, y=1, title = "Tweets by week",labels={'0':'Week of Month','1':'Number of Tweets'}, range_y=(5000,12000))
fig.show()

##Retweets and Likes per week of Month

In [16]:
totalsPerWeek = []
total = 0
ototal = 0
x=0
y=7
num=1
#While not past the last day of the month
while y<36:
    #Select tweets likes between (start date) and (end date) of the week of the month
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #For each number of likes from week
    for counts in res:
        #Add likes to total for the week
        total = total+counts['favorites_count']

    #Select tweets retweets between (start date) and (end date) of the week of the month
    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #For each number of retweets from week
    for retweets in res:
        #Add retweets to total for the week
        ototal = ototal+retweets['retweet_count']
    #Add both retweets and likes per week totals to a list
    totalsPerWeek.append([num,total,ototal])
    x=y+1
    #If third week add 10 to the end date to capture all values for the last week
    if(y==21):
        y=y+10
    #Otherwise update by 7 to get to next week
    else:
        y=y+7
    #Update week number
    num=num+1
    ototal=0
    total=0

print(totalsPerWeek)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]


In [17]:
retweetweek = []
likesweek = []
x=1
#For each value in the totals list
for i in totalsPerWeek:
    #Seperate into retweets and likes individual lists
    retweetweek.append([x,i[2]])
    likesweek.append([x,i[1]])
    #Update x
    x=x+1

In [18]:
#Chart retweets by week of month
fig = px.bar(retweetweek, x=0, y=1, title = "retweets by week of month",labels={'0':'Week of Month','1':'Retweet per Tweet'}, range_y=(0,30000000))
fig.show()

In [19]:
#Chart likes by week of month
fig = px.bar(likesweek, x=0, y=1, title = "Likes by week of month",labels={'0':'Week of Month','1':'Like per Tweet'}, range_y=(0,50000000))
fig.show()

##Retweets per Tweet and Likes per Tweet per Week of Month

In [20]:
print(totalsPerWeek)
print(tweetsPerWeek)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]
[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [21]:
x = 1
avgRetweetpw = []
avgLikespw = []
#For each week of month (1-4)
while x < 5:
    #Take total retweets of the week and divide by total tweets made per week
    average = (totalsPerWeek[x-1][2])/(tweetsPerWeek[x-1][1])
    #Add to retweet per tweet per week list
    avgRetweetpw.append([x,average])
    #Take total likes of the week and divde by total tweets made per week
    average = (totalsPerWeek[x-1][1])/(tweetsPerWeek[x-1][1])
    #Add to likes per tweet per week list
    avgLikespw.append([x,average])
    #Update x
    x=x+1

print(avgRetweetpw)
print(avgLikespw)

[[1, 2424.3321765953347], [2, 1458.4412679219208], [3, 1494.1590859865714], [4, 1726.8659510519535]]
[[1, 3566.2895700790436], [2, 2947.621350837796], [3, 3801.8975525232836], [4, 4172.268441391155]]


In [22]:
#Chart retweets per tweet by week of month (1-4)
fig = px.bar(avgRetweetpw, x=0, y=1, title = "retweet per tweet by week of month",labels={'0':'week of month','1':'Retweets per Tweet'}, range_y=(0,4000))
fig.show()

In [23]:
#Chart likes per tweet by week of month (1-4)
fig = px.bar(avgLikespw, x=0, y=1, title = "like per tweet by week of month",labels={'0':'week of month','1':'Like per Tweet'}, range_y=(0,5000))
fig.show()

#T-Testing

##Per Weekday

In [32]:
import numpy as np
from scipy import stats
import re
sum = 0
x = 1
y = 1
totalOther = []
totalDay = []
totalDay2 = []
totalOther2 = []
resultsLikes=[]
resultsRetweets=[]
#Running for each day of week
while y<8:
    #Running 7 times
    while x < 8:
        #Select likes where dow = x
        query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
        cursor.execute(query) #Executing query
        res = cursor.fetchall()
        #For each number of likes
        for i in res:
            #If not weekday
            if(x!=y):
                #Add to other list
                totalOther.append(i['favorites_count'])
            else:
                #If weekday...Add to day list
                totalDay.append(i['favorites_count'])

        #Select retweets where dow = x
        query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
        cursor.execute(query) #Executing query
        res = cursor.fetchall()
        #For each number of retweets
        for i in res:
            #If not weekday
            if(x!=y):
                #Add to other list
                totalOther2.append(i['retweet_count'])
            else:
                #If weekday...Add to day list
                totalDay2.append(i['retweet_count'])
        x=x+1
    #Adding results to approproate list
    resultsLikes.append(str([y,stats.ttest_ind(totalOther, totalDay)]))
    resultsRetweets.append(str([y,stats.ttest_ind(totalOther2, totalDay2)]))
    #Resetting lists
    totalOther = []
    totalDay = []
    totalOther2 = []
    totalDay2 = []
    #Updating days and y
    x=1
    y=y+1



print(resultsLikes)
print(resultsRetweets)


#running a t-test
#If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null
#hypothesis of identical average scores. The null hypothesis here is that the retweet/likes counts averages are the same
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
#then we reject the null hypothesis of equal averages.

#If the p-value is small (Less than 0.05) and the T-stat is negative,
#it means that it is probable that the likes/retweets average of the singular day is higher than the other days

['TtestResult(statistic=2.6647033197546555, pvalue=0.007708514095540831, df=42829.0)', 'TtestResult(statistic=0.8919595056465808, pvalue=0.3724196487993565, df=42829.0)', 'TtestResult(statistic=0.7837005184302288, pvalue=0.43322019212565066, df=42829.0)', 'TtestResult(statistic=1.5351399731603927, pvalue=0.1247568360203621, df=42829.0)', 'TtestResult(statistic=-2.0366349872579903, pvalue=0.04169277141028242, df=42829.0)', 'TtestResult(statistic=-1.0850702927188118, pvalue=0.2778966311258495, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)']
['TtestResult(statistic=1.9098304005343465, pvalue=0.05616172668248298, df=42829.0)', 'TtestResult(statistic=1.2350236637963525, pvalue=0.21682847237631958, df=42829.0)', 'TtestResult(statistic=0.4328120825221578, pvalue=0.6651534742038784, df=42829.0)', 'TtestResult(statistic=0.2923721947806585, pvalue=0.7700034814982757, df=42829.0)', 'TtestResult(statistic=0.4228914715798581, pvalue=0.6723765686

##Per Week of Month

In [39]:
mtotalOther = []
mtotalWeek = []
mtotalOther2 = []
mtotalWeek2 = []
x=0
y=7
sum=0
#Running while not past last week
while y<36:
    #Selecting likes where week is (1-4)
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #For each likes number
    for counts in res:
        #If not in the (enter number for week)
        if(x!=22):
            #Add to other list
            mtotalOther.append(counts['favorites_count'])
        else:
            #if in week, add to week list
            mtotalWeek.append(counts['favorites_count'])


    #Selecting retweets where week is (1-4)
    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #For each retweets number
    for num in res:
        #If not in the (enter number for week)
        if(x!=22):
            #Add to other list
            mtotalOther2.append(num['retweet_count'])
        else:
            #if in week, add to week list
            mtotalWeek2.append(num['retweet_count'])

    #Update week
    x=y+1
    #If third week add 10 to the end date to capture all values for the last week
    if(y==21):
        y=y+10
    #Otherwise update by 7 to get to next week
    else:
        y=y+7




print(str(stats.ttest_ind(mtotalOther, mtotalWeek)))
print(str(stats.ttest_ind(mtotalOther2, mtotalWeek2)))

#running a t-test
#If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null
#hypothesis of identical average scores. The null hypothesis here is that the retweet/likes counts averages are the same
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
#then we reject the null hypothesis of equal averages.

#If the p-value is small (Less than 0.05) and the T-stat is negative,
#it means that it is probable that the likes/retweets average of the singular week is higher than the other weeks

TtestResult(statistic=-2.7476253697687216, pvalue=0.006005365820634144, df=42829.0)
TtestResult(statistic=0.2293955109568773, pvalue=0.8185626202846767, df=42829.0)


#When does Elon Musk Tweet?

In [86]:
x = 1
weekdayCountElon=[]
totalElon = 0
ototalElon = 0
#Going through all 7 days (1 Monday to 7 Sunday)
while x < 8:
    #Select tweets made by elon musk on day = 8
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE user_id = '44196397' AND extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #Add to day and count list
    weekdayCountElon.append([x,res[0]['count']])
    #Update x
    x=x+1


print(weekdayCountList)
#Chart Elon Tweets by Weekday
import plotly.express as px
fig = px.bar(weekdayCountElon, x=0, y=1, title = "Tweets by weekday",labels={'0':'Day of Week','1':'Number of Tweets'})
fig.show()


[[1, 76], [2, 70], [3, 77], [4, 98], [5, 72], [6, 81], [7, 97]]


In [174]:
x = 1
weekCountElon=[]
totalElon = 0
ototalElon = 0
y=7
num=0
#Going thrpugh all four weeks
while y < 36:
    #Select Elon's count of tweets from (start date) to (end date)
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE user_id = '44196397' AND extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    #Add to day and count list
    weekCountElon.append([num,res[0]['count']])
    #update dates
    x=y+1
    #If third week add 10 to get all dates to catch all final dates
    if(y==21):
        y=y+10
    #Otherwise add 7 to go to next week
    else:
        y=y+7
    #Update week number
    num=num+1

print(weekCountElon)
#Chart Elon's Tweets by Week
import plotly.express as px
fig = px.bar(weekCountElon, x=0, y=1, title = "Elon's Tweets by week of month",labels={'0':'Week of Month','1':'Number of Tweets'})
fig.show()

[[0, 151], [1, 105], [2, 133], [3, 182]]


#Putting It All Together

In [165]:
#Selecting for likes from best week (last week) and best dow (5)
query = "SELECT favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '5' AND extract(day FROM created_date) BETWEEN '22' and '32'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
bestTweets = []
#Add likes to best tweets list
for x in res:
    bestTweets.append(x['favorites_count'])

#Selecting for likes NOT from best week (last week) and best dow (5)
query = "SELECT favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) != '5' AND extract(day FROM created_date) NOT BETWEEN '22' and '32'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
otherTweets = []
#Add likes to other tweets list
for x in res:
    otherTweets.append(x['favorites_count'])

print(str(stats.ttest_ind(otherTweets, bestTweets)))

TtestResult(statistic=-4.398376407348794, pvalue=1.0947066550226364e-05, df=27659.0)


In [163]:
#Selecting for retweets from best week (last week) and best dow (5)
query = "SELECT retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '6' AND extract(day FROM created_date) BETWEEN '1' and '7'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
bestTweets = []
#Add retweets to best tweets list
for x in res:
    bestTweets.append(x['retweet_count'])

#Selecting for retweets NOT from best week (last week) and best dow (5)
query = "SELECT retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) != '6' AND extract(day FROM created_date) NOT BETWEEN '1' and '7'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
otherTweets = []
#Add retweets to other tweets list
for x in res:
    otherTweets.append(x['retweet_count'])

print(str(stats.ttest_ind(otherTweets, bestTweets)))

TtestResult(statistic=-5.168866635358705, pvalue=2.3703694891158435e-07, df=29787.0)
